# Pipeline prolijo: métricas de clientes y target futuro

Este notebook crea dos artefactos principales:
1) `df_train_metrics`: métricas por cliente calculadas usando órdenes hasta la fecha de referencia 2024-06-01 (sin target).
2) `df_train_with_target`: `df_train_metrics` con la columna `target` derivada del conteo de órdenes en la ventana futura 2024-06-01 → 2024-09-30 (target=1 si NO hay órdenes en esa ventana).

Todas las operaciones son no destructivas respecto a `df_orders` original: se crean copias filtradas y agregadas.


In [ ]:
# Imports y opciones de visualización
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)


In [ ]:
# Leer los CSVs (rutas relativas a la carpeta del repo)
df_clientes = pd.read_csv('data/clients_202510280926.csv')
df_orders = pd.read_csv('data/orders_202510280929.csv')
df_products = pd.read_csv('data/products_202510280931.csv')

# Vista rápida
print('clientes:', df_clientes.shape, 'orders:', df_orders.shape, 'products:', df_products.shape)
df_clientes.head()


In [ ]:
# Crear df_orders_products sin modificar df_orders original
df_orders_products = df_orders.merge(df_products, left_on='product_id', right_on='product_id', how='left', suffixes=('_order','_product'))

# Asegurar tipo fecha
df_orders_products['order_timestamp'] = pd.to_datetime(df_orders_products['order_timestamp'])

print('df_orders_products filas:', df_orders_products.shape[0])
df_orders_products.head()


In [ ]:
# Función helper: construir métricas por cliente dadas órdenes y fecha de referencia
def build_customer_metrics(df, fecha_ref):
    df = df.copy()
    df['order_timestamp'] = pd.to_datetime(df['order_timestamp'])
    # Fecha de referencia como Timestamp
    fecha_ref = pd.to_datetime(fecha_ref)

    # Categoria favorita por número de órdenes
    fav_category = (
        df.groupby(['client_id', 'product_category']).size()
        .reset_index(name='count')
        .sort_values(['client_id', 'count'], ascending=[True, False])
        .drop_duplicates('client_id')
        .rename(columns={'product_category': 'favorite_category'})
    )

    # Agregados principales
    agg = df.groupby('client_id').agg(
        total_orders=('order_timestamp','count'),
        first_order_date=('order_timestamp','min'),
        last_order_date=('order_timestamp','max'),
        total_spent=('order_price','sum'),
        average_order_value=('order_price','mean'),
        std_order_value=('order_price','std'),
        max_order_value=('order_price','max'),
        min_order_value=('order_price','min'),
        total_shipping_cost=('shipping_cost','sum'),
        avg_shipping_cost=('shipping_cost','mean'),
        unique_categories=('product_category','nunique'),
        unique_brands=('brand','nunique'),
        unique_payment_methods=('payment_method','nunique'),
        unique_shipping_methods=('shipping_method','nunique'),
        cancelled_orders=('order_status', lambda x: (x=='failed').sum()),
        express_shipping_ratio=('shipping_method', lambda x: x.str.contains('express', case=False).mean() if x.notna().any() else 0),
        most_used_payment_method=('payment_method', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
        most_used_shipping_method=('shipping_method', lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    ).reset_index()

    # Features temporales respecto a fecha_ref
    agg['days_since_last_order'] = (fecha_ref - agg['last_order_date']).dt.days
    agg['customer_lifespan_days'] = (fecha_ref - agg['first_order_date']).dt.days
    agg['avg_days_between_orders'] = agg['customer_lifespan_days'] / agg['total_orders']
    agg['delay_vs_avg_interval'] = agg['days_since_last_order'] - agg['avg_days_between_orders']

    # Meses activos
    df['month'] = df['order_timestamp'].dt.to_period('M')
    months_active = df.groupby('client_id')['month'].nunique().reset_index(name='months_active')
    agg = agg.merge(months_active, on='client_id', how='left')

    # Categoria favorita y gasto en ella
    agg = agg.merge(fav_category[['client_id','favorite_category']], on='client_id', how='left')
    spend_by_cat = df.groupby(['client_id','product_category'])['order_price'].sum().reset_index()
    spend_fav = spend_by_cat.merge(fav_category[['client_id','favorite_category']], on='client_id')
    spend_fav = spend_fav[spend_fav['product_category'] == spend_fav['favorite_category']]
    spend_fav = spend_fav.rename(columns={'order_price':'favorite_category_spend'})
    agg = agg.merge(spend_fav[['client_id','favorite_category_spend']], on='client_id', how='left')
    agg['fav_category_spend_ratio'] = agg['favorite_category_spend'] / agg['total_spent']

    # Reemplazar NaNs sensatos
    agg.fillna({
        'std_order_value': 0,
        'favorite_category_spend': 0,
        'fav_category_spend_ratio': 0,
        'months_active': 0
    }, inplace=True)

    # Ordenar columnas (si existen)
    cols = [
        'client_id','total_orders','first_order_date','last_order_date',
        'days_since_last_order','customer_lifespan_days','avg_days_between_orders','delay_vs_avg_interval',
        'total_spent','average_order_value','std_order_value','max_order_value','min_order_value',
        'total_shipping_cost','avg_shipping_cost','unique_categories','unique_brands',
        'unique_payment_methods','unique_shipping_methods','cancelled_orders','express_shipping_ratio',
        'months_active','most_used_payment_method','most_used_shipping_method',
        'favorite_category','favorite_category_spend','fav_category_spend_ratio'
    ]
    agg = agg[[c for c in cols if c in agg.columns]]
    return agg


In [ ]:
# 1) df_train: órdenes hasta 2024-06-01 (inclusive) y cálculo de métricas
fecha_ref_train = '2024-06-01'
df_train = df_orders_products[df_orders_products['order_timestamp'] <= pd.to_datetime(fecha_ref_train)].reset_index(drop=True)
print('Órdenes en df_train:', df_train.shape[0])
df_train_metrics = build_customer_metrics(df_train, fecha_ref_train)
print('Clientes con métricas (train):', df_train_metrics.shape[0])
df_train_metrics.head()


In [ ]:
# 2) df_future: ventana 2024-06-01 → 2024-09-30 (inclusive) y cálculo del target
start_future = pd.to_datetime('2024-06-01')
end_future = pd.to_datetime('2024-09-30')
df_future = df_orders_products[(df_orders_products['order_timestamp'] >= start_future) & (df_orders_products['order_timestamp'] <= end_future)].reset_index(drop=True)
print('Órdenes en ventana futura:', df_future.shape[0])
# Contar órdenes por cliente en la ventana futura
df_future_counts = df_future.groupby('client_id').size().reset_index(name='future_orders')
# Merge con todos los clientes de df_train_metrics para obtener target para cada cliente entrenable
df_train_with_target = df_train_metrics.merge(df_future_counts, on='client_id', how='left')
# Clientes sin registros en df_future_counts tienen 0 órdenes en la ventana futura => target=1
df_train_with_target['future_orders'] = df_train_with_target['future_orders'].fillna(0).astype(int)
df_train_with_target['target'] = (df_train_with_target['future_orders'] == 0).astype(int)
print('Distribución target (1 = no compró en la ventana futura):')
print(df_train_with_target['target'].value_counts(dropna=False).to_string())
df_train_with_target.head()


In [ ]:

print('Notebook generado: df_train_metrics y df_train_with_target disponibles en memoria.')
